In [3]:
!pip install matplotlib torch scipy librosa cn2an opencc

In [ ]:
pip install torch==2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

hps = utils.get_hparams_from_file("./configs/cantonese_base.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./models/G.pth", net_g, None)

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
INFO:root:dec.cond.weight is not in the checkpoint
INFO:root:dec.cond.bias is not in the checkpoint
INFO:root:enc_q.enc.cond_layer.bias is not in the checkpoint
INFO:root:enc_q.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:enc_q.enc.cond_layer.weight_v is not in the checkpoint
INFO:root:flow.flows.0.enc.cond_layer.bias is not in the checkpoint
INFO:root:flow.flows.0.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:flow.flows.0.enc.cond_layer.weight_v is not in the checkpoint
INFO:root:flow.flows.2.enc.cond_layer.bias is not in the checkpoint
INFO:root:flow.flows.2.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:flow.flows.2.enc.cond_layer.weight_v is not in the checkpoint
INFO:root:flow.flows.4.enc.cond_layer.bias is not in the checkpoint
INFO:root:flow.flows.4.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:flow.flows.4.enc.cond_layer.weight_v 

In [14]:
import opencc
t2s = opencc.OpenCC('t2s.json')
input_text = t2s.convert("我愛你")

stn_tst = get_text("[GD]" + input_text + "[GD]", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1.2)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [ ]:
# save audio
import soundfile as sf
sf.write('audio.mp3', audio, 22050, format='mp3')